In [ ]:
from flask import Flask #flask에서 Flask라는 클래스 import
import urllib.request

app = Flask(__name__) #Flask라는 클래스의 객체를 생성(app), 인수: __name__

@app.route("/tospring", methods=['GET']) #생성한 객체의 route 설정(=URL 설정)
def spring():
    return marko_json

if __name__ == '__main__': #객체의 run함수를 이용해서 로컬 서버에서 어플리케이션 실행
    app.run(debug=False, host="127.0.0.1", port=5000)

In [4]:
# !pip install import_ipynb
# !pip install pykrx
# !pip install -U finance-datareader
# !pip install openpyxl
# !pip install pyyaml

# # Markowitz
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull

from datetime import datetime, timedelta, date
from pykrx import stock

import FinanceDataReader as web

# # Auto Trading & Alert
# import requests
# import time
# import yaml

# # Recommend
# import getpass
# import openpyxl
# from urllib.request import urlopen

import import_ipynb

In [5]:
num_stock = 3
stock_name = ["삼성전자", "카카오", "대한항공"]

In [7]:
n = int(input("숫자 입력(0: Recommend, 1: Markowitz, 2: Vol_auto, 3: Port_auto, 4: stock_alert): "))
if n == 0:
    import Recommend as R
    print(R.recommend_json)
elif n == 1:
    # 데이터 가져오기

    def stock_reader(kospi_df, stock_name, code_list, df='p', n=0):

        if n == 0:
            n = len(code_list)

        stock_df = pd.DataFrame()
        stock_df['Date'] = kospi_df['Date']
        print("동기간 KOSPI 생성일수 : ", len(kospi_df['Date']))
        normal_cnt = 0
        err_cnt = 0
        code_nm_list = []
        symbol_list = []

        print("n", n)

        for code in code_list:
            stock = df_krx[df_krx.Code == code]
            code_nm = list(stock.Name)[0]
            symbol = list(stock.Code)[0]


            try:
                temp = web.DataReader(code, start_date, end_date)

                # 데이터일수가 시장보다 작으면 skip(최근 상장 데이터로 판단)
                if len(temp) < len(kospi_df['Date']):
                    err_cnt += 1
                    print("skip : (",err_cnt,")", code_list[code], code_nm, start_date, end_date, ", 건수 : ", len(temp))
                    continue

                temp.reset_index(inplace = True)
                temp_df = pd.merge(temp[['Date','Close', 'Open']], kospi_df[['Date']], on='Date', how='right')

                if df == 'v':
                    stock_df[code_nm] = temp_df.Close - temp_df.Open #variation
                elif df == 'p':
                    stock_df[code_nm] = temp_df.Close # price

                normal_cnt += 1
                code_nm_list.append(code_nm)
                symbol_list.append(symbol)
                print("정상 : (",normal_cnt,")", code, code_nm, start_date, end_date, ", 건수 : ", len(temp), "->", len(stock_df))
            except:
                err_cnt += 1
                print("skip : (",err_cnt,")", code, code_nm, start_date, end_date, ", 건수 : ", len(temp))

            if normal_cnt == n:
                print('총', n,'개 생성 설정 / ', normal_cnt, '개 생성 완료')
                break # n개 종목 생성시 종료

            # 데이터 정렬
            stock_df.sort_values('Date', ascending=True, inplace=True) # ascending=True 오름차순, False 내림차순
            # 결측치 채우기
            stock_df.fillna(method='ffill', inplace=True)

        return stock_df, code_nm_list, symbol_list

    code_list = [] # 종목 코드 리스트
    tickers = stock.get_market_ticker_list(market="KOSPI")
    for i in range(num_stock):
        for ticker in tickers:
            if stock_name[i] == stock.get_market_ticker_name(ticker):
                code_list.append(ticker)
    
    start_date = date.today() - timedelta(days=366)
    end_date = date.today() - timedelta(days=1)

    # 한국 거래소 상장종목 전체
    # 용도 : 코드와 종목명 가져오기
    df_krx = web.StockListing('KRX')

    kospi_df = web.DataReader('KS11', start_date, end_date)
    kospi_df.reset_index(inplace = True)
    
    stock_df, code_nm_list, symbol_list = stock_reader(kospi_df, stock_name, code_list, 'p', num_stock) # v :variation, p : price(close)
    
    stock_df.dropna(inplace=True)
    data = stock_df.set_index('Date')
    
    # 현재 포트폴리오 설정 (랜덤 Weight로 산출)
    num_assets = np.size(code_nm_list)
    cur_value = (1e4-5e3)*np.random.rand(num_assets,1) + 5e3        
    tot_value = np.sum(cur_value)
    weights = cur_value.ravel()/float(tot_value)
    
    #compute portfolio risk
    Sigma = data.cov().values
    Corr = data.corr().values

    volatility = np.sqrt(np.dot(weights.T, np.dot(Sigma, weights)))
    
    weight_list = weights.tolist()

    marko_dict = {"markowitz":{"stk_nm":num_stock, "stk_cd":code_list, "stk_wt":weight_list}}
    
    marko_json = json.dumps(marko_dict, indent=4)
    
    print(marko_json)
elif n == 2:
    import VolatilityStockAutoTrade as V
    print(V.VAT_json)
elif n == 3:
    import PortfolioStockAutoTrade as P
    print(P.PAT_json)
elif n == 4:
    import stock_alert as A
    print(A.alert_json)
else:
    print("에러!")

숫자 입력(0: Recommend, 1: Markowitz, 2: Vol_auto, 3: Port_auto, 4: stock_alert): 1
동기간 KOSPI 생성일수 :  245
n 3
정상 : ( 1 ) 005930 삼성전자 2021-11-15 2022-11-15 , 건수 :  248 -> 245
정상 : ( 2 ) 035720 카카오 2021-11-15 2022-11-15 , 건수 :  248 -> 245
정상 : ( 3 ) 003490 대한항공 2021-11-15 2022-11-15 , 건수 :  248 -> 245
총 3 개 생성 설정 /  3 개 생성 완료
{
    "markowitz": {
        "stk_nm": 3,
        "stk_cd": [
            "005930",
            "035720",
            "003490"
        ],
        "stk_wt": [
            0.4171597347539247,
            0.2991216199095051,
            0.28371864533657026
        ]
    }
}


In [ ]:
app = Flask(__name__) #Flask라는 클래스의 객체를 생성(app), 인수: __name__

@app.route("/tospring", methods=['POST']) #생성한 객체의 route 설정(=URL 설정)
def spring():
    return marko_json

if __name__ == '__main__': #객체의 run함수를 이용해서 로컬 서버에서 어플리케이션 실행
    app.run(debug=False, host="127.0.0.1", port=5000)